<a href="https://colab.research.google.com/github/RashikaTyagi01/Rashika-UML501/blob/main/mlAssignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit) Download the dataset regarding USA House Price Prediction from the following link: https://drive.google.c om/file/d/1O_Nw pJT-8xGfU_-3llUl2sgPu0xllOrX/view ?usp=sharing Load the dataset and Implement 5- fold cross validation for multiple linear regression (using least square error fit).
Steps:
a) Divide the dataset into input features (all columns except price) and output variable
(price)
b) Scale the values of input features.
c) Divide input and output features into five folds.
d) Run five iterations, in each iteration consider one-fold as test set and remaining
four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score
for each iteration using least square error fit.
e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the
regressor for 70% of data and test the performance for remaining 30% data.

In [28]:
import pandas as pd
import numpy as np

In [29]:
data = pd.read_csv('/USA_Housing.csv')
data.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [30]:
#a
X = data.drop('Price',axis=1)
y = data['Price']

In [32]:
#b
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [34]:
#c,d,e
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
betas = []
r2_scores = []
for train_idx, test_idx in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train_with_intercept = np.c_[np.ones(len(X_train)), X_train]
    X_test_with_intercept = np.c_[np.ones(len(X_test)), X_test]
    beta = np.linalg.inv(X_train_with_intercept.T @ X_train_with_intercept) @ (X_train_with_intercept.T @ y_train)
    betas.append(beta)
    y_pred = X_test_with_intercept @ beta
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
best_index = np.argmax(r2_scores)
best_beta = betas[best_index]
best_r2 = r2_scores[best_index]
print("R2 scores:", r2_scores)
print("Best R2 score:", best_r2)
print("Best beta matrix:\n", best_beta)

X_train70, X_test30, y_train70, y_test30 = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train70_with_intercept = np.c_[np.ones(len(X_train70)), X_train70]
X_test30_with_intercept = np.c_[np.ones(len(X_test30)), X_test30]
final_beta = np.linalg.inv(X_train70_with_intercept.T @ X_train70_with_intercept) @ (X_train70_with_intercept.T @ y_train70)
y_pred30 = X_test30_with_intercept @ final_beta
final_r2 = r2_score(y_test30, y_pred30)
print("final beta matrix:\n", final_beta)
print("R2 score for 30% test data:", final_r2)

R2 scores: [0.9179971706985147, 0.9145677884802819, 0.9116116385364478, 0.9193091764960817, 0.9243869413350317]
Best R2 score: 0.9243869413350317
Best beta matrix:
 [1.23161736e+06 2.30225051e+05 1.63956839e+05 1.21115120e+05
 7.83467170e+02 1.50662447e+05]
final beta matrix:
 [1231278.63687691  230464.52520478  164159.19982569  120514.71328324
    2913.62424674  151019.35865134]
R2 score for 30% test data: 0.9146818498916266


Concept of Validation set for Multiple Linear Regression (Gradient Descent
Optimization)
Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the dataset into training set (56%), validation set (14%), and test set (30%).
Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of regression coefficients for each value of learning rate after 1000 iterations.
For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regression coefficients.

In [35]:

df = pd.read_csv('/USA_Housing.csv')
X = df.drop('Price',axis=1)
y = df['Price']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = np.c_[np.ones(len(X_scaled)), X_scaled]
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.44, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=30/44, random_state=42)

def gradient_descent(X, y, alpha, iterations):
    m, n = X.shape
    beta = np.zeros(n)
    for i in range(iterations):
        y_pred = X @ beta
        gradient = (1/m) * (X.T @ (y_pred - y))
        beta -= alpha * gradient
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
best_lr = None
best_beta = None
best_r2_val = -np.inf
best_r2_test = None

for lr in learning_rates:
    beta = gradient_descent(X_train, y_train, lr, 1000)
    y_val_pred = X_val @ beta
    r2_val = r2_score(y_val, y_val_pred)
    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_lr = lr
        best_beta = beta
        best_r2_test = r2_score(y_test, X_test @ beta)

print("Best Learning Rate:", best_lr)
print("Best validation R2:", best_r2_val)
print("Test R2 with best beta:", best_r2_test)
print("Best beta coefficients:\n", best_beta)



Best Learning Rate: 0.1
Best validation R2: 0.9199649194854793
Test R2 with best beta: 0.9134494051887397
Best beta coefficients:
 [1232180.27200919  230645.88389435  165328.94019375  120045.00851908
    2945.02108903  151375.22971285]


Pre-processing and Multiple Linear Regression
Download the dataset regarding Car Price Prediction from the following link:
https://arc hive.ic s.uci.edu/ml/mac hine-learning-databases/autos/imports-85.data
1. Load the dataset with following column names ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"] and replace all ? values with NaN
2. Replace all NaN values with central tendency imputation. Drop the rows with NaN values in price column
3. There are 10 columns in the dataset with non-numeric values. Convert these values to numeric values using following scheme:
(i) For “num_doors” and “num_cylinders”: convert words (number names) to figures
for e.g., two to 2
(ii) For "body_style", "drive_wheels": use dummy encoding scheme
(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding
scheme
(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.
(v) For engine_type: replace values containing string ohc to 1 else all values to 0.
4. Divide the dataset into input features (all columns except price) and output variable (price). Scale all input features.
5. Train a linear regressor on 70% of data (using inbuilt linear regression function of Python) and test its performance on remaining 30% of data.
6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then again train a linear regressor on 70% of reduced data (using inbuilt linear regression function of Python). Does it lead to any performance improvement on test set?

In [36]:
#1,2
cols = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
        "num_doors", "body_style", "drive_wheels", "engine_location",
        "wheel_base", "length", "width", "height", "curb_weight",
        "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore",
        "stroke", "compression_ratio", "horsepower", "peak_rpm",
        "city_mpg", "highway_mpg", "price"]

df1 = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",names=cols)
df1.replace("?",np.nan,inplace=True)
df1.dropna(subset=["price"],inplace=True)
for col in df1.columns:
    if df1[col].dtype == 'object':  # categorical
        df1[col] = df1[col].fillna(df1[col].mode()[0])
    else:
        df1[col] = df1[col].astype(float).fillna(df1[col].astype(float).mean())
print(df1.isnull().sum())

symboling            0
normalized_losses    0
make                 0
fuel_type            0
aspiration           0
num_doors            0
body_style           0
drive_wheels         0
engine_location      0
wheel_base           0
length               0
width                0
height               0
curb_weight          0
engine_type          0
num_cylinders        0
engine_size          0
fuel_system          0
bore                 0
stroke               0
compression_ratio    0
horsepower           0
peak_rpm             0
city_mpg             0
highway_mpg          0
price                0
dtype: int64


In [37]:
#3(i),(ii)
from sklearn.preprocessing import LabelEncoder

word_to_num = { "two": 2,"three": 3,"four": 4, "five": 5,"six": 6, "eight": 8,"twelve": 12}
df1["num_doors"] = df1["num_doors"].replace(word_to_num)
df1["num_cylinders"] = df1["num_cylinders"].replace(word_to_num)
df1['num_doors'] = df1['num_doors'].fillna(df1['num_doors'].mode()[0])

df1 = pd.get_dummies(df1, columns=["body_style", "drive_wheels"], drop_first=True)

print(df1.head())

   symboling normalized_losses         make fuel_type aspiration  num_doors  \
0        3.0               161  alfa-romero       gas        std          2   
1        3.0               161  alfa-romero       gas        std          2   
2        1.0               161  alfa-romero       gas        std          2   
3        2.0               164         audi       gas        std          4   
4        2.0               164         audi       gas        std          4   

  engine_location  wheel_base  length  width  ...  peak_rpm  city_mpg  \
0           front        88.6   168.8   64.1  ...      5000      21.0   
1           front        88.6   168.8   64.1  ...      5000      21.0   
2           front        94.5   171.2   65.5  ...      5000      19.0   
3           front        99.8   176.6   66.2  ...      5500      24.0   
4           front        99.4   176.6   66.4  ...      5500      18.0   

  highway_mpg  price  body_style_hardtop body_style_hatchback  \
0        27.0  13495 

/tmp/ipython-input-2856605374.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1["num_doors"] = df1["num_doors"].replace(word_to_num)
/tmp/ipython-input-2856605374.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1["num_cylinders"] = df1["num_cylinders"].replace(word_to_num)


In [38]:
#3(iii)
from sklearn.preprocessing import LabelEncoder
c = ["make", "aspiration", "engine_location", "fuel_type"]
l = LabelEncoder()
for col in c:
    df1[col] = l.fit_transform(df1[col])

In [39]:
#3(iv),(v)
df1['fuel_system'] = df1['fuel_system'].apply(lambda x: 1 if 'pfi' in str(x).lower() else 0)
df1['engine_type'] = df1['engine_type'].apply(lambda x: 1 if 'ohc' in str(x).lower() else 0)

In [40]:
#4
X1=df1.drop(columns=["price"]).values
y1=df1["price"].values
scaler = StandardScaler()
X1 = scaler.fit_transform(X1)


In [41]:
#5
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=42)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("R2 score: ",r2)
print("mean squared error:",mse)



R2 score:  0.8680078106543789
mean squared error: 12511758.50198219


In [42]:
#6
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X)
Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)
lr_pca = LinearRegression()
lr_pca.fit(Xp_train, yp_train)
yp_pred = lr_pca.predict(Xp_test)
print("With PCA:\n")
print("R2 Score:", r2_score(yp_test, yp_pred))
print("mse:", mean_squared_error(yp_test, yp_pred))

With PCA:

R2 Score: 0.5594558941914749
mse: 51988752725.36713
